In [ ]:
import kaggle

In [ ]:
import tensorflow as tf

In [ ]:
import keras

In [ ]:
#Importing kaggle-hosted dataset "New Plant Diseases Dataset" 
#Requires a Kaggle API Authenification
#https://www.kaggle.com/vipoooool/new-plant-diseases-dataset

!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

In [ ]:
#Unzip and download data.  
#IMPORTANT: This is a large file of 87K images.  Do not execute if you do not with to download to your current directory
from zipfile import ZipFile
zf = ZipFile('new-plant-diseases-dataset.zip')
zf.extractall()
zf.close()


In [ ]:
#We will load images for Keras to interpret with the Image Data Generator.
#All images are scaled by 1:255 to "transform every pixel value from range [0,255] -> [0,1]".  See https://www.linkedin.com/pulse/keras-image-preprocessing-scaling-pixels-training-adwin-jahn/

from tensorflow.keras.preprocessing.image import ImageDataGenerator
gen_trn = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest')
gen_valtst = ImageDataGenerator(rescale = 1./255)


In [ ]:
# load and iterate train data
train_batch = gen_trn.flow_from_directory('New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/', class_mode='categorical', batch_size=64, seed=12, target_size = (128,128))
# Load and iterate val data
val_batch = gen_valtst.flow_from_directory('New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/', class_mode='categorical', batch_size=64, seed=123,  target_size = (128,128))



In [ ]:
#Examine assigned class indices:
train_batch.class_indices
classes = train_batch.class_indices
classes


In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils.vis_utils import plot_model


In [ ]:
#For L1 regularization import:
from keras.regularizers import l1


In [ ]:
#Creating Non-TL model (A):

full_model = Sequential()
#CNN
full_model.add(Convolution2D(32, (3, 3), input_shape = (128,128,3), activation = 'relu'))
#Pooling
full_model.add(MaxPooling2D(pool_size=(2,2)))
#Flatting to 1D for Dense layers
full_model.add(Flatten())
full_model.add(Dense(units=512,activation='relu'))
full_model.add(Dense(units=256,activation='relu'))
#output layer with L1 regularization
full_model.add(Dense(units=38,activation='softmax',activity_regularizer=l1(0.001)))

#Compile Non-TL Model with Adam optimization
full_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(full_model.summary())


In [ ]:
#Keep track of processing time:
from time import process_time

t1_start = process_time() 


#Fit model
fm_fit = full_model.fit_generator(
    train_batch,
    steps_per_epoch = 150,
    epochs = 10,
    validation_data = val_batch,
    validation_steps = 50
    )


# Stop the stopwatch / counter 
t1_stop = process_time() 
   
print("Elapsed time:", round((t1_stop-t1_start)/360,2),"m")


In [ ]:
#Transfer Learning
from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Convolution2D, MaxPooling2D,Dropout


In [ ]:
r_model = Sequential()
r_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
r_model.add(Dense(38, activation='softmax'))
r_model.compile(optimizer = 'adam', loss= 'categorical_crossentropy', metrics = ['accuracy'])

#This model is to show how ResNet50 automatically improves accuracy by providing pre-trained image classification info. 
#However, we did not add any additional learning, just the output softmax layer, so of course the accuracy did not improve much after the first epoch.
print(r_model.summary())


In [ ]:
#Fitting the ResNet50 + output layer model:

t1_start = process_time()  
r_fit = full_model.fit_generator(
    train_batch,
    steps_per_epoch = 300,
    epochs = 10,
    validation_data = val_batch,
    validation_steps = 50
    )


# Stop the stopwatch / counter 
t1_stop = process_time() 
   
print("Elapsed time:", round((t1_stop-t1_start)/360,2),"m")


In [ ]:
#Now comparing ResNet50 results to EfficientNetB0

import efficientnet.keras as efn
Using TensorFlow backend.



In [ ]:
#EfficientnetB0 model: method adapted from #adapted from: https://www.kaggle.com/ateplyuk/keras-starter-efficientnet

eff_net = efn.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

eff_net.trainable = False


from keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense,Dropout
from keras import Model

x = eff_net.output
x = Conv2D(32, (3, 3), input_shape = (128,128,3), activation = 'sigmoid', padding = "same")(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(64, (3, 3), input_shape = (128,128,3), activation = 'sigmoid', padding = "same")(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Flatten()(x)
x = Dense(512, activation=tf.nn.swish)(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
predictions2a = Dense(38, activation="softmax")(x)
model_e_cnn2a = Model(input = eff_net.input, output = predictions2)


In [ ]:
model_e_cnn2a.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_e_cnn2a.summary()


In [ ]:
#CNN efficientnet model

from time import process_time
t1_start = process_time()  
ecnn_fit = model_e_cnn2a.fit_generator(
    train_batch,
    steps_per_epoch = 100,
    epochs = 10,
    validation_data = val_batch,
    validation_steps = 10
    )


# Stop the stopwatch / counter 
t1_stop = process_time() 
   
print("Elapsed time:", round((t1_stop-t1_start)/360,2),"m")
